# LSTM IMDB Movie Review Tutorial
Josiah Olson

In [1]:
from __future__ import print_function
import time
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, merge, BatchNormalization
from keras.datasets import imdb

import os
from keras.preprocessing.text import Tokenizer

Using Theano backend.


In [2]:
max_features = 10000
max_len = 200  # cut texts after this number of words (among top max_features most common words)

In [3]:
# get dataset and unzip: http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

X_train = []
y_train = []

path = 'C:/1_Research/Create_data/aclImdb/train_vs_23/pos/'
X_train.extend([open(path + f).read() for f in os.listdir(path) if f.endswith('.txt')])
y_train.extend([1 for _ in range(10554)])

path = 'C:/1_Research/Create_data/aclImdb/train_vs_23/neg/'
X_train.extend([open(path + f).read() for f in os.listdir(path) if f.endswith('.txt')])
y_train.extend([0 for _ in range(3665)])

print('x:')
print(X_train[:1])
print(X_train[-1:])
print(len(X_train))
print('y:')
print(y_train[:1])
print(y_train[-1:])
print(len(y_train))

x:
[' ipo benthanh tourist nh\xc3\xa0 \xc4\x91\xe1\xba\xa7u t\xc6\xb0 \xc4\x91\xc4\x83ng k mua tri\xe1\xbb\x87u cp g\xe1\xba\xa5p l\xe1\xba\xa7n s\xe1\xbb\x91 c\xe1\xbb\x95 ph\xe1\xba\xa7n \xc4\x91\xe1\xba\xa5u gi\xc3\xa1 vietstock c\xe1\xbb\xa5 th\xe1\xbb\x83 nh\xc3\xa0 \xc4\x91\xe1\xba\xa7u t\xc6\xb0 c\xc3\xa1 nh\xc3\xa2n trong n\xc6\xb0\xe1\xbb\x9bc \xc4\x91\xc4\x83ng k mua tri\xe1\xbb\x87u cp v\xc3\xa0 nh\xc3\xa0 \xc4\x91\xe1\xba\xa7u t\xc6\xb0 t\xe1\xbb\x95 ch\xe1\xbb\xa9c \xc4\x91\xc4\x83ng k mua h\xc6\xa1n tri\xe1\xbb\x87u cp benthanh tourist \xc4\x91\xc3\xa1ng ch\xc3\xba c\xc3\xb3 m\xe1\xbb\x99t nh\xc3\xa0 \xc4\x91\xe1\xba\xa7u t\xc6\xb0 c\xc3\xa1 nh\xc3\xa2n n\xc6\xb0\xe1\xbb\x9bc ngo\xc3\xa0i \xc4\x91\xc4\x83ng k mua h\xc6\xa1n tri\xe1\xbb\x87u cp benthanh tourist b\xe1\xba\xb1ng v\xe1\xbb\x9bi s\xe1\xbb\x91 c\xe1\xbb\x95 ph\xe1\xba\xa7n m\xc3\xa0 doanh nghi\xe1\xbb\x87p n\xc3\xa0y s\xe1\xba\xbd \xc4\x91\xe1\xba\xa5u gi\xc3\xa1 v\xc3\xa0o ng\xc3\xa0y t\xe1\xbb\x9bi c\xe1\xbb\

In [4]:
# read in the test data

X_test = []
y_test = []

path = 'C:/1_Research/Create_data/aclImdb/test_vs_23/pos/'
X_test.extend([open(path + f).read() for f in os.listdir(path) if f.endswith('.txt')])
y_test.extend([1 for _ in range(5277)])

path = 'C:/1_Research/Create_data/aclImdb/test_vs_23/neg/'
X_test.extend([open(path + f).read() for f in os.listdir(path) if f.endswith('.txt')])
y_test.extend([0 for _ in range(1832)])

print('x:')
print(X_test[:1])
print(X_test[-1:])
print(len(X_test))
print('y:')
print(y_test[:1])
print(y_test[-1:])
print(len(y_test))

x:
[' ngh\xe1\xba\xbdn gi\xe1\xba\xa3i ng\xc3\xa2n v\xe1\xbb\x91n oda vietstock th\xc3\xb4ng tin t\xe1\xbb\xab v\xc4\x83n ph\xc3\xb2ng ch\xc3\xadnh ph\xe1\xbb\xa7 cho bi\xe1\xba\xbft t\xe1\xba\xa1i cu\xe1\xbb\x99c h\xe1\xbb\x8dp ban ch\xe1\xbb\x89 \xc4\x91\xe1\xba\xa1o qu\xe1\xbb\x91c gia v\xe1\xbb\x81 oda v\xc3\xa0 v\xe1\xbb\x91n vay \xc6\xb0u \xc4\x91\xc3\xa3i ng\xc3\xa0y ph\xc3\xb3 th\xe1\xbb\xa7 t\xc6\xb0\xe1\xbb\x9bng ph\xe1\xba\xa1m b\xc3\xacnh minh tr\xc6\xb0\xe1\xbb\x9fng ban ch\xe1\xbb\x89 \xc4\x91\xe1\xba\xa1o \xc4\x91\xc3\xa3 y\xc3\xaau c\xe1\xba\xa7u b\xe1\xbb\x99 k\xe1\xba\xbf ho\xe1\xba\xa1ch v\xc3\xa0 \xc4\x91\xe1\xba\xa7u t\xc6\xb0 x\xc3\xa1c \xc4\x91\xe1\xbb\x8bnh c\xc3\xa1c l\xc4\xa9nh v\xe1\xbb\xb1c c\xe1\xba\xa7n \xc6\xb0u ti\xc3\xaan v\xc3\xa0 t\xc3\xacm c\xc3\xa1ch \xc4\x91\xe1\xba\xa9y nhanh t\xe1\xbb\x91c \xc4\x91\xe1\xbb\x99 gi\xe1\xba\xa3i ng\xc3\xa2n ngu\xe1\xbb\x93n v\xe1\xbb\x91n oda theo h\xc6\xb0\xe1\xbb\x9bng kh\xc3\xb4ng t\xc4\x83ng th\xc3\xaam c\xc3\xa

In [5]:
#tokenize works to list of integers where each integer is a key to a word
imdbTokenizer = Tokenizer(nb_words=max_features)

imdbTokenizer.fit_on_texts(X_train)

In [6]:
#print top 20 words 
#note zero is reserved for non frequent words
for word, value in imdbTokenizer.word_index.items():
    if value < 20:
        print(value, word)

3 trong
6 có
1 và
18 đã
13 đầu
4 đồng
12 công
5 các
14 được
2 của
16 nbsp
17 tăng
9 năm
11 cho
8 tỷ
19 doanh
15 cổ
7 với
10 là


In [7]:
#create int to word dictionary
intToWord = {}
for word, value in imdbTokenizer.word_index.items():
    intToWord[value] = word

#add a symbol for null placeholder
intToWord[0] = "!!!NA!!!"
    
print(intToWord[1])
print(intToWord[2])
print(intToWord[32])

và
của
tại


In [8]:
#convert word strings to integer sequence lists
print(X_train[0])
print(imdbTokenizer.texts_to_sequences(X_train[:1]))
for value in imdbTokenizer.texts_to_sequences(X_train[:1])[0]:
    print(intToWord[value])
    
X_train = imdbTokenizer.texts_to_sequences(X_train)
X_test = imdbTokenizer.texts_to_sequences(X_test)

 ipo benthanh tourist nhà đầu tư đăng k mua triệu cp gấp lần số cổ phần đấu giá vietstock cụ thể nhà đầu tư cá nhân trong nước đăng k mua triệu cp và nhà đầu tư tổ chức đăng k mua hơn triệu cp benthanh tourist đáng chú có một nhà đầu tư cá nhân nước ngoài đăng k mua hơn triệu cp benthanh tourist bằng với số cổ phần mà doanh nghiệp này sẽ đấu giá vào ngày tới cụ thể nhà đầu tư cá nhân trong nước đăng k mua triệu cp và nhà đầu tư tổ chức đăng k mua hơn triệu cp benthanh tourist đáng chú có một nhà đầu tư cá nhân nước ngoài đăng k mua hơn triệu cp benthanh tourist bằng với số cổ phần mà doanh nghiệp này sẽ đấu giá vào ngày tới 
[[1032, 5764, 5235, 33, 13, 21, 332, 184, 101, 135, 118, 511, 209, 29, 15, 119, 591, 20, 202, 315, 61, 33, 13, 21, 359, 141, 3, 64, 332, 184, 101, 135, 118, 1, 33, 13, 21, 222, 157, 332, 184, 101, 39, 135, 118, 5764, 5235, 400, 577, 6, 34, 33, 13, 21, 359, 141, 64, 174, 332, 184, 101, 39, 135, 118, 5764, 5235, 229, 7, 29, 15, 119, 207, 19, 78, 28, 26, 591, 20, 42, 

In [9]:
# Censor the data by having a max review length (in number of words)

#use this function to load data from keras pickle instead of munging as shown above
#(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features,
#                                                      test_split=0.2)

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print("Pad sequences (samples x time)")
X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, maxlen=max_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
y_train = np.array(y_train)
y_test = np.array(y_test)

14219 train sequences
7109 test sequences
Pad sequences (samples x time)
X_train shape: (14219L, 200L)
X_test shape: (7109L, 200L)


In [10]:
#example of a sentence sequence, note that lower integers are words that occur more commonly
print("x:", X_train[0]) #per observation vector of 20000 words
print("y:", y_train[0]) #positive or negative review encoding

x: [   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0 1032 5764 5235
   33   13   21  332  184  101  135  118  511  209   29   15  119  591   20
  202  315   61   33   13   21  359  141    3   64  332  184  101  135  118
    1   33   13   21  222  157  332  184  101   39  135  118 5764 5235  400
  577    6   34   33   13   21  359  141   64  174  332  184  101   39  135
  118 5764 5235  229    7   29   15  119  207   19   78   28   26  591   20
   42   59  178  315   61   33   13   21  359  141    3   64  332  184  101
  135  118    1   33   13   21  222  157  332  184  101   39  135  118 5764
 5235  400  577    6   34   33   13   21  359  141   64  174  332  184  101
   39  135  118 5764 5235  229    7   29   15  119  207   19   78   28   26
  591   2

In [11]:
# double check that word sequences behave/final dimensions are as expected
print("y distribution:", np.unique(y_train, return_counts=True))
print("max x word:", np.max(X_train), "; min x word", np.min(X_train))
print("y distribution test:", np.unique(y_test, return_counts=True))
print("max x word test:", np.max(X_test), "; min x word", np.min(X_test))

y distribution: (array([0, 1]), array([ 3665, 10554], dtype=int64))
max x word: 9998 ; min x word 0
y distribution test: (array([0, 1]), array([1832, 5277], dtype=int64))
max x word test: 9997 ; min x word 0


In [12]:
print("most and least popular words: ")
print(np.unique(X_train, return_counts=True))
# as expected zero is the highly used word for words not in index

most and least popular words: 
(array([   0,    1,    2, ..., 9994, 9995, 9998]), array([421173,  30008,  26791, ...,      2,      1,      1], dtype=int64))


In [13]:
#set model hyper parameters
epochs = 6
embedding_neurons = 128
lstm_neurons = 64
batch_size = 32

Next let's import pre-trained word vectors from google and use them to initialize our embedding to see if this improves the neural net's performance.

In [14]:
from gensim.models import Word2Vec

#get pre trained word2vec from google:
#https://doc-0k-4g-docs.googleusercontent.com/docs/securesc/gnqvgap6hjncpd3b10i2tv865io48jas/hmjtdgee48c14e1parufukrpkb8urra5/1463018400000/06848720943842814915/09676831593570546402/0B7XkCwpI5KDYNlNUTTlSS21pQmM?e=download&nonce=4l49745nmtine&user=09676831593570546402&hash=i2qe9mshan4mesl112ct9bu1tj9kr1hq

googlew2v = Word2Vec.load_word2vec_format('C:/1_Research/Create_data/aclImdb/word2vector/GoogleNews-vectors-negative300.bin', binary=True)

C:\Anaconda2\lib\site-packages\gensim\utils.py:840: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Anaconda2\lib\site-packages\gensim\utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [15]:
# get word vectors for words in my index
googleVecs = []
for value in range(max_features):
    try:
        googleVecs.append(googlew2v[intToWord[value]])
    except:
        googleVecs.append(np.random.uniform(size=300))

googleVecs = np.array(googleVecs)

print(googleVecs)
print(googleVecs.shape)

[[ 0.26202468  0.15868397  0.27812652 ...,  0.10937801  0.75767728
   0.4428527 ]
 [ 0.96970407  0.12253514  0.35782362 ...,  0.97491444  0.6667405
   0.40817497]
 [ 0.34841225  0.36222138  0.90096367 ...,  0.47904751  0.85001801
   0.84470638]
 ..., 
 [ 0.28022387  0.40691114  0.9637814  ...,  0.26572361  0.91806214
   0.82371557]
 [ 0.25881884  0.58101921  0.58019771 ...,  0.23012591  0.28700454
   0.05928751]
 [ 0.38440905  0.53545783  0.36813549 ...,  0.79370321  0.9866407
   0.84512547]]
(10000L, 300L)


C:\Anaconda2\lib\site-packages\gensim\models\word2vec.py:1578: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  return self.syn0[self.vocab[words].index]


In [16]:
# Bi-directional google

# this example tests if using pretrained embeddings will improve performance 
# relative to starting with random embeddings

# this is the placeholder tensor for the input sequences
sequence = Input(shape=(max_len,), dtype='int32')
# this embedding layer will transform the sequences of integers
# into vectors of size embedding
# embedding layer converts dense int input to one-hot in real time to save memory
embedded = Embedding(max_features, 300, input_length=max_len, weights=[googleVecs])(sequence)
# normalize embeddings by input/word in sentence
bnorm = BatchNormalization()(embedded)

# apply forwards LSTM layer size lstm_neurons
forwards = LSTM(lstm_neurons, dropout_W=0.4, dropout_U=0.4)(bnorm)
# apply backwards LSTM
backwards = LSTM(lstm_neurons, dropout_W=0.4, dropout_U=0.4, go_backwards=True)(bnorm)

# concatenate the outputs of the 2 LSTMs
merged = merge([forwards, backwards], mode='concat', concat_axis=-1)
after_dp = Dropout(0.5)(merged)
output = Dense(1, activation='sigmoid')(after_dp)

model_bidir_google = Model(input=sequence, output=output)
# review model structure
print(model_bidir_google.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 200)           0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 200, 300)      3000000     input_1[0][0]                    
____________________________________________________________________________________________________
batchnormalization_1 (BatchNormal(None, 200, 300)      600         embedding_1[0][0]                
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 64)            93440       batchnormalization_1[0][0]       
___________________________________________________________________________________________

In [17]:
# Bi-directional google

model_bidir_google.compile('rmsprop', 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
start_time = time.time()

history_bidir_google = model_bidir_google.fit(X_train, y_train,
                    batch_size=batch_size,
                    nb_epoch=epochs,
                    validation_data=[X_test, y_test], 
                    verbose=2)

end_time = time.time()
average_time_per_epoch = (end_time - start_time) / epochs
print("avg sec per epoch:", average_time_per_epoch)

Train...
Train on 14219 samples, validate on 7109 samples
Epoch 1/6
847s - loss: 0.5984 - acc: 0.7285 - val_loss: 0.5848 - val_acc: 0.7423
Epoch 2/6
875s - loss: 0.5721 - acc: 0.7415 - val_loss: 0.5933 - val_acc: 0.7423
Epoch 3/6
888s - loss: 0.5639 - acc: 0.7425 - val_loss: 0.5934 - val_acc: 0.7423
Epoch 4/6
812s - loss: 0.5489 - acc: 0.7429 - val_loss: 0.6080 - val_acc: 0.7422
Epoch 5/6
718s - loss: 0.5411 - acc: 0.7435 - val_loss: 0.6300 - val_acc: 0.7405
Epoch 6/6
683s - loss: 0.5273 - acc: 0.7512 - val_loss: 0.6478 - val_acc: 0.7381
avg sec per epoch: 808.967166662
